In [1]:
import numpy as np
import pandas as pd

from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import f1_score
from sklearn.metrics import accuracy_score
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
import requests

from tqdm import tqdm

In [2]:
data = pd.read_csv('NLP_data_list_0715.csv',encoding='latin1')
data.head()

,file,description,size,link
0,train_set.csv.zip,ÑµÁ·¼¯Êý¾Ý£¨Ñ¡ÊÖÐèÒª×ÔÐÐ½âÑ¹£©,236.11MB,https://tianchi-competition.oss-cn-hangzhou.al...
1,test_a.csv.zip,²âÊÔ¼¯A°ñÊý¾Ý£¨Ñ¡ÊÖÐèÒª×ÔÐÐ½âÑ¹£©,59.12MB,https://tianchi-competition.oss-cn-hangzhou.al...
2,test_a_sample_submit.csv,²âÊÔ¼¯A°ñÌá½»ÑùÀý,97.66KB,https://tianchi-competition.oss-cn-hangzhou.al...


In [3]:
data['link']

0    https://tianchi-competition.oss-cn-hangzhou.al...
1    https://tianchi-competition.oss-cn-hangzhou.al...
2    https://tianchi-competition.oss-cn-hangzhou.al...
Name: link, dtype: object

In [ ]:
train_link = data['link'][0]
test_link = data['link'][1]
submit_link = data['link'][2]
print(train_link)
print(test_link)
print(submit_link)

In [5]:
train = pd.read_csv('train_set.csv', sep = '\t')
test = pd.read_csv('test_a.csv',sep = '\t')
train.head()

,label,text
0,2,2967 6758 339 2021 1854 3731 4109 3792 4149 15...
1,11,4464 486 6352 5619 2465 4802 1452 3137 5778 54...
2,3,7346 4068 5074 3747 5681 6093 1777 2226 7354 6...
3,2,7159 948 4866 2109 5520 2490 211 3956 5520 549...
4,3,3646 3055 3055 2490 4659 6065 3370 5814 2465 5...


In [7]:
train_text = train['text']
test_text = test['text']
all_text = pd.concat([train_text, test_text])

In [9]:
%%time
word_vectorizer = TfidfVectorizer(
    sublinear_tf=True,
    strip_accents='unicode',
    analyzer='word',
    token_pattern=r'\w{1,}',
    stop_words='english',
    ngram_range=(1, 1),
    max_features=10000)

word_vectorizer.fit(all_text)
train_word_features = word_vectorizer.transform(train_text)
test_word_features = word_vectorizer.transform(test_text)
train_word_features

CPU times: user 7min 49s, sys: 30.2 s, total: 8min 19s
Wall time: 8min 58s


<200000x6977 sparse matrix of type '<class 'numpy.float64'>'
	with 56074040 stored elements in Compressed Sparse Row format>

In [10]:
X_train = train_word_features
y_train = train['label']

# 可以改变输入维度
x_train_, x_valid_, y_train_, y_valid_ = train_test_split(X_train, y_train, test_size=0.2)
X_test = test_word_features

In [11]:
%%time
clf = LogisticRegression(C=4, n_jobs=16)
clf.fit(x_train_, y_train_)

y_pred = clf.predict(x_valid_)
train_scores = clf.score(x_train_, y_train_)
print(train_scores, f1_score(y_pred, y_valid_, average='macro'))

0.94910625 0.921020202773586
CPU times: user 2.02 s, sys: 8.18 s, total: 10.2 s
Wall time: 5min 27s


In [ ]:
class XGB():
    
    def __init__(self, X_df, y_df):
        self.X = X_df
        self.y = y_df
       
    def train(self, param):
        self.model = XGBClassifier(**param)
        self.model.fit(self.X, self.y, eval_set=[(self.X, self.y)],
                       eval_metric=['mlogloss'],
                       early_stopping_rounds=10,  # 连续N次分值不再优化则提前停止
                       verbose=False
                      )
        
#         mode evaluation
        train_result, train_proba = self.model.predict(self.X), self.model.predict_proba(self.X)
        train_acc = accuracy_score(self.y, train_result)
        train_auc = f1_score(self.y, train_proba, average='macro')
        
        print("Train acc: %.2f%% Train auc: %.2f" % (train_acc*100.0, train_auc))
        
    def test(self, X_test, y_test):
        result, proba = self.model.predict(X_test), self.model.predict_proba(X_test)
        acc = accuracy_score(y_test, result)
        f1 = f1_score(y_test, proba, average='macro')
        
        print("acc: %.2f%% F1_score: %.2f%%" % (acc*100.0, f1))
    
    def grid(self, param_grid):
        self.param_grid = param_grid
        xgb_model = XGBClassifier(nthread=20)
        clf = GridSearchCV(xgb_model, self.param_grid, scoring='f1_macro', cv=2, verbose=1)
        clf.fit(self.X, self.y)
        print("Best score: %f using parms: %s" % (clf.best_score_, clf.best_params_))
        return clf.best_params_, clf.best_score_



In [ ]:
 x_train_, x_valid_, y_train_, y_valid_ = train_test_split(X_train[:, :300], y_train, test_size=0.2, shuffle=True, random_state=42)
X_test = test_word_features[:,:300]

In [ ]:
param = {'learning_rate': 0.05,         #  (xgb’s “eta”)
              'objective': 'multi:softmax', 
              'n_jobs': 16,
              'n_estimators': 300,           # 树的个数
              'max_depth': 10,               
              'gamma': 0.5,                  # 惩罚项中叶子结点个数前的参数，Increasing this value will make model more conservative.
              'reg_alpha': 0,               # L1 regularization term on weights.Increasing this value will make model more conservative.
              'reg_lambda': 2,              # L2 regularization term on weights.Increasing this value will make model more conservative.
              'min_child_weight' : 1,      # 叶子节点最小权重
              'subsample':0.8,             # 随机选择80%样本建立决策树
              'random_state':1           # 随机数
             }
model = XGB(x_train_, y_train_)
model.train(param)
model.test(x_valid_, y_valid_)

In [ ]:
final_model = XGB(X_train, y_train)
final_model.train(param)

submission = pd.read_csv('./datalab/72510/test_a_sample_submit.csv')
preds = final_model.model.predict(X_test)
submission['label'] = preds
submission.to_csv('./xgb_submission.csv', index=False)